In [1]:
from datasets import hennepin_rebase

dataset = hennepin_rebase.dataset_hennepin_rebase(mode='test', sample_mode='')



In [2]:
import torch

print(dataset[0][0].shape)
masks = dataset[0][1]
output = torch.ones((1,302,302))

output = torch.flatten(output,start_dim = 1)
masks = torch.flatten(masks,start_dim = 1)

region_sums = torch.matmul(output, masks.T.float())
print(region_sums)
print(region_sums.shape)
print(dataset[0][2])


torch.Size([3, 302, 302])
tensor([[1154., 1951., 1760., 1244., 1383., 1178., 1341., 2794., 2267., 2800.,
         1406., 1497.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.]])
torch.Size([1, 100])
[405500. 487000. 494600. 360700. 430800. 413400. 454000. 527600. 501000.
 533000. 376900. 444400.      0.      0.      0.      0.      0.      0.
      0.      0

In [3]:
import torch
import pytorch_lightning as pl
from models import unet

In [4]:
import pytorch_lightning as pl
from models import unet
# Linear Basline
class RALModule(pl.LightningModule):
    def __init__(self,use_pretrained):
        super().__init__()
        self.unet = unet.UNet(in_channels=3, out_channels=1)

        if(use_pretrained):
            pretrained_state = torch.load('/u/eag-d1/data/Hennepin/model_checkpoints/building_seg_pretrained.pth')

            model_state = self.unet.state_dict()
            pretrained_state = { k:v for k,v in pretrained_state.items() if k in model_state and v.size() == model_state[k].size() }
            model_state.update(pretrained_state)
            self.unet.load_state_dict(model_state)

        self.loss_torch = torch.nn.MSELoss()

    def forward(self, x):
        x = self.unet(x)
        x = torch.flatten(x, start_dim=2)
        return x

    def shared_step(self, batch):
        image, masks, values = batch

        output = self(image)

        masks = torch.flatten(masks,start_dim = 2)
        masks = torch.swapdims(masks,2,1)

        #print(output.shape)
        #print(masks.shape)
    
        region_sums_yhat = torch.matmul(output.float(), masks.float()).squeeze(1)

        #print(region_sums_yhat[0])
        #print(values[0])

        print(region_sums_yhat.shape)
        print(values.shape)
        # loss
        #loss = self.loss_torch(region_sums_yhat, values.float())
        squares = torch.square(values.float()-region_sums_yhat)
        loss = torch.sum(squares, dim=1).mean()
        #print(loss)
        #loss = torch.tensor(0).float()
        
        return {'loss': loss}

    def training_step(self, batch, batch_idx):
        output = self.shared_step(batch)
        self.log('train_loss', output['loss'], on_epoch = True)
        return output['loss']

    # def validation_step(self, batch, batch_idx):
    #     output = self.shared_step(batch)
    #     self.log('val_loss', output['loss'], on_epoch = True)
    #     return output['loss']

    def test_step(self, batch, batch_idx):
        output = self.shared_step(batch)
        self.log('test_loss', output['loss'], on_epoch = True)
        return output['loss']

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [5]:
train = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=False, num_workers=4)

In [6]:
model = RALModule(use_pretrained=True)
trainer = pl.Trainer(gpus=[0], max_epochs = 100)
trainer.fit(model, train)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name       | Type    | Params
---------------------------------------
0 | unet       | UNet    | 13.4 M
1 | loss_torch | MSELoss | 0     
---------------------------------------
13.4 M    Trainable params
0         Non-trainable params
13.4 M    Total params
53.581    Total estimated model params size (MB)


Epoch 0:   0%|          | 0/136 [00:00<?, ?it/s] tensor([-155.4597,   -3.4164,   37.9252,  -44.6831,  -13.6402,    9.0595,
         -36.3991,  124.3349,   38.4357,  459.8157,   50.8364,  119.6926,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000, 

/localdisk0/SCRATCH/cgarchbold/envs/py39/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
